In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.utils.data as Data

seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

epochs= 100
learning_rate = 1e-4

from PIL import  Image,ImageDraw
import numpy as np


data1 = []
data2= []

for i in range(20000):
    img = Image.new('RGB', (32, 32), color = 'white')

    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')

    draw = ImageDraw.Draw(im2)

    draw.rectangle([21,12,24,15],fill='red',outline='red')
    data2.append(np.asarray(im2))


for i in range(20000):
    img = Image.new('RGB', (32, 32), color = 'white')


    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')

    draw = ImageDraw.Draw(im2)

    draw.rectangle([9,12,12,15],fill='red',outline='red')
    data1.append(np.asarray(im2))
    

y0 = np.array(data1, dtype=np.float32)





y1 = np.array(data2, dtype=np.float32)
y0 = np.transpose(y0, (0, 3, 2, 1))# This to make channels first for CNN
y1 = np.transpose(y1, (0, 3, 2, 1)) 

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)



torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=250,

    shuffle=True,

    num_workers=0,

)


class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 1)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

#  use gpu if available

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = CNN()






# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
# mean-squared error loss
def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h

for epoch in range(epochs):
    loss = 0
    for x,y in loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        #print(batch_features[0].shape)
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        # compute reconstructions        
        # compute training reconstruction loss
        train_loss = h_score(model(x),model(y))
        #train_loss = criterion(model(x),model(y))
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs,loss))
    #print(np.shape(model.encoder_hidden_layer.weight.data),np.shape(model_dec.decoder_output_layer.weight.data))




epoch : 1/100, recon loss = -0.00002365
epoch : 2/100, recon loss = 0.00020252
epoch : 3/100, recon loss = -0.00017057
epoch : 4/100, recon loss = -0.00009494
epoch : 5/100, recon loss = -0.00066796
epoch : 6/100, recon loss = -0.00921049
epoch : 7/100, recon loss = -0.10014464
epoch : 8/100, recon loss = -0.35598918
epoch : 9/100, recon loss = -0.60739362
epoch : 10/100, recon loss = -0.73042363
epoch : 11/100, recon loss = -0.80245209
epoch : 12/100, recon loss = -0.83976209
epoch : 13/100, recon loss = -0.84654919
epoch : 14/100, recon loss = -0.86663714
epoch : 15/100, recon loss = -0.88098449
epoch : 16/100, recon loss = -0.88396214
epoch : 17/100, recon loss = -0.88787583
epoch : 18/100, recon loss = -0.89451275
epoch : 19/100, recon loss = -0.89702724
epoch : 20/100, recon loss = -0.90332046
epoch : 21/100, recon loss = -0.90130295
epoch : 22/100, recon loss = -0.89995291
epoch : 23/100, recon loss = -0.90901962
epoch : 24/100, recon loss = -0.90964098
epoch : 25/100, recon loss

In [ ]:
blank = cv2.imread('blank_28x28.png')
b = blank.astype('float32')
b = torch.from_numpy(b)

for i in range(5,15):
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))

print("b:", model(b.reshape(1,3,28,28)))